In [2]:
from conf import databases, url, data_dir, architecture_1
from file_utils import create_folder
from nn import LasagneNN

import numpy
import random
import time

import lasagne
from matplotlib import pyplot as plt
import matplotlib as mpl

In [5]:
# print(databases)
# only = ['mitdb', 'ltdb', 'ltafdb', 'ltstdb']
# dbs = [(db, ids) for (db, ids) in databases if db in only]
# print(dbs)

[('mitdb', ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']), ('ltdb', ['14046', '14134', '14149', '14157', '14172', '14184', '15814']), ('ltafdb', ['00', '01', '03', '05', '06', '07', '08', '10', '100', '101', '102', '103', '104', '105', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '13', '15', '16', '17', '18', '19', '20', '200', '201', '202', '03', '204', '205', '206', '207', '208', '21', '22', '23', '24', '25', '26', '28', '30', '32', '33', '34', '35', '37', '38', '39', '42', '43', '44', '45', '47', '48', '49', '51', '53', '54', '55', '56', '58', '60', '62', '64', '65', '68', '69', '70', '71', '72', '74', '75']), ('ltstdb', ['s20011', 's20021'

In [3]:
fs = 128
fs_target = 360
min_bpm = 10
max_bpm = 350
min_gap = fs*60/min_bpm
max_gap = fs*60/max_bpm

train_perc = 99
test_perc = 100 - train_perc

params = {
    'fs_target': 360,
    'segment_size': 5000, 
    'segment_step': 2500, 
    'normalize_steps': True,
    'correct_peaks': True,
    'left_border': 50, # samples
    'right_border': 50, # samples
    'min_gap': min_gap,
    'max_gap': max_gap
}

In [4]:
model_path = 'saved-models/' + '_'.join(only)
create_folder(model_path)
save_name = model_path + time.strftime("%Y-%m-%d-%H-%M")

In [ ]:
nn = NN(architecture_1, dim=1, params=params)

Model shape:
  Layer A:
    a1: (None, 8, 5000)
    a2: (None, 64, 5000)
    a3: (None, 64, 5000)
    a4: (None, 32, 5000)
    a5: (None, 16, 5000)
    a6: (None, 16, 5000)
    a7: (None, 16, 5000)
   al: (None, 216, 5000)
  Layer B:
    b1: (None, 8, 5000)
    b2: (None, 64, 5000)
    b3: (None, 64, 5000)
    b4: (None, 64, 5000)
   bl: (None, 136, 5000)
  Layer C:
    c1: (None, 8, 5000)
    c2: (None, 32, 5000)
    c3: (None, 32, 5000)
    c4: (None, 32, 5000)
    c5: (None, 32, 5000)
   cl: (None, 136, 5000)
  Layer D:
    d1: (None, 8, 5000)
    d2: (None, 32, 5000)
    d3: (None, 64, 5000)
   dl: (None, 104, 5000)
  Later E:
    e1: (None, 16, 5000)
    e2: (None, 16, 5000)
    e3: (None, 16, 5000)
   el: (None, 48, 5000)
  Layer F:
    f1: (None, 8, 5000)
    f2: (None, 16, 5000)
    f3: (None, 16, 5000)
   fl: (None, 40, 5000)
  Layer G:
    g1: (None, 1, 5000)
   gl: (None, 1, 5000)


In [ ]:
exs = shuffled_examples(databases, 360, 5000, 2500, True, True)
train_exs = exs[:int(len(exs)*train_perc/100)]
test_exs  = exs[int(len(exs)*train_perc/100):]
print('Number of examples: {} ({} samples)'.format(len(exs), len(exs)*5000))
print('          Training: {} ({} samples)'.format(len(train_exs), len(train_exs)*5000))
print('           Testing: {} ({} samples)'.format(len(test_exs), len(test_exs)*5000))

In [ ]:
nn.train(train_exs, test_exs, num_epochs=500, examples_by_epoch=1000, info=True, model_path=model_path)

In [ ]:
_, params = load_model(model_path + '/model-loss0.037631392673427445-epoch1200.sav')
# print(params)
set_all_param_values(nn.model, params)
nn.init_funs(nn.model)

In [ ]:
eval_model(testXY, nn.evaluate, left_border=left_border, right_border=right_border,
           min_gap=min_gap, max_gap=max_gap, plot_examples=True, nb=10,
           nearest_fpr=0.000675, threshold=0.9678147, eval_margin=10)